# 🚀 Superise Backend - Sürekli Çalışan Versiyon

## Kullanım:
1. Runtime > Change runtime type > GPU seçin
2. Bu hücreyi çalıştırın
3. Verilen public URL'i kullanarak dışarıdan istekler atabilirsiniz

## API Endpoint'leri:
- GET /docs - Swagger UI
- POST /enhance - Görüntü iyileştirme

In [ ]:
#@title 🔄 Sürekli Çalışan Superise Backend {display-mode: "form"}

#@markdown ## 📋 Ayarlar
#@markdown Ngrok token'ınızı girin (https://dashboard.ngrok.com/get-started/your-authtoken)
ngrok_token = '' #@param {type:"string"}

import os
import time
import IPython
from IPython.display import clear_output, display, HTML
import subprocess
import sys

def install_requirements():
    print("📦 Gerekli paketler yükleniyor...")
    !pip install -q pyngrok nest_asyncio fastapi uvicorn
    # Özel sürüm gereksinimleri
    !pip install torch==2.0.1 torchvision==0.15.2
    !pip install basicsr==1.4.2 facexlib==0.2.5 gfpgan==1.3.5
    clear_output()

def setup_gpu():
    print("🔍 GPU kontrol ediliyor...")
    import torch
    if not torch.cuda.is_available():
        raise Exception("❌ GPU bulunamadı! Runtime > Change runtime type > GPU seçin")
    print(f"✅ GPU hazır: {torch.cuda.get_device_name(0)}")

def clone_repo():
    print("📥 Proje indiriliyor...")
    !git clone -q https://github.com/mehmetcanyildiz/superise-backend
    %cd superise-backend
    
    # basicsr kütüphanesindeki import hatasını düzelt
    !sed -i 's/from torchvision.transforms.functional_tensor import rgb_to_grayscale/from torchvision.transforms.functional import rgb_to_grayscale/' /usr/local/lib/python3.10/dist-packages/basicsr/data/degradations.py
    
    clear_output()

def setup_ngrok(token):
    from pyngrok import ngrok, conf
    if token:
        print("🔑 Ngrok ayarlanıyor...")
        !ngrok authtoken $token
    return ngrok.connect(8000)

def keep_alive():
    display(HTML('''
    <script>
        function ClickConnect(){
            console.log("Bağlantı korunuyor...");
            document.querySelector("colab-connect-button").click()
        }
        setInterval(ClickConnect, 60000)
    </script>
    '''))

def display_status(public_url):
    display(HTML(f'''
    <div style="padding: 20px; border-radius: 10px; background-color: #f0f9ff; margin: 10px 0;">
        <h3 style="color: #0369a1;">🌐 Superise Backend Çalışıyor!</h3>
        <p><b>Public URL:</b> <a href="{public_url}" target="_blank">{public_url}</a></p>
        <p><b>API Docs:</b> <a href="{public_url}/docs" target="_blank">{public_url}/docs</a></p>
        <p><b>Örnek curl komutu:</b></p>
        <code style="background: #e0f2fe; padding: 10px; display: block; border-radius: 5px;">
        curl -X POST \
             -H "Content-Type: multipart/form-data" \
             -F "image=@resim.jpg" \
             {public_url}/enhance
        </code>
    </div>
    '''))

def run_server():
    import nest_asyncio
    nest_asyncio.apply()
    !python run_uvicorn.py

def main():
    try:
        install_requirements()
        setup_gpu()
        
        if not os.path.exists('superise-backend'):
            clone_repo()
        
        tunnel = setup_ngrok(ngrok_token)
        keep_alive()
        display_status(tunnel.public_url)
        
        run_server()
        
    except Exception as e:
        print(f"❌ Hata: {str(e)}")
        raise

if __name__ == "__main__":
    main()